In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.0 MB/s eta 0:00:00


In [2]:
import transformers

In [3]:
print(transformers.__version__)

4.33.2


In [14]:
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification
from transformers import TextClassificationPipeline

import tensorflow as tf
import pandas as pd
import json
import gc

from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopw = stopwords.words('english')

import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot

from tqdm import tqdm



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
root_path = '../content/bbc-text.csv'

In [19]:
df = pd.read_csv(root_path)

In [20]:
df.head()

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...


In [23]:
df.shape

(2225, 2)

In [24]:
df['count'] = df['text'].apply(lambda x: len(x.split()))
df.head()

,category,text,count
0,tech,tv future in the hands of viewers with home th...,737
1,business,worldcom boss left books alone former worldc...,300
2,sport,tigers wary of farrell gamble leicester say ...,246
3,sport,yeading face newcastle in fa cup premiership s...,341
4,entertainment,ocean s twelve raids box office ocean s twelve...,260


In [25]:
category_count = df['category'].value_counts()
categories = category_count.index
categories

Index(['sport', 'business', 'politics', 'tech', 'entertainment'], dtype='object')

In [26]:
category_count.index

Index(['sport', 'business', 'politics', 'tech', 'entertainment'], dtype='object')

In [27]:
df['encoded_text'] = df['category'].astype('category').cat.codes

In [28]:
df.head()

,category,text,count,encoded_text
0,tech,tv future in the hands of viewers with home th...,737,4
1,business,worldcom boss left books alone former worldc...,300,0
2,sport,tigers wary of farrell gamble leicester say ...,246,3
3,sport,yeading face newcastle in fa cup premiership s...,341,3
4,entertainment,ocean s twelve raids box office ocean s twelve...,260,1


In [29]:
data_texts = df['text'].to_list()
data_labels = df['encoded_text'].to_list()

In [30]:
train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size= 0.2)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.1)

In [34]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', num_labels = 5)
train_encodings = tokenizer(train_texts, truncation = True, padding = True)
val_encodings = tokenizer(val_texts, truncation = True, padding = True)

In [35]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels
))

In [36]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 5)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [39]:
from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments

training_args = TFTrainingArguments(
    output_dir = './results',
    num_train_epochs = 1,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 64,
    warmup_steps = 500,
    weight_decay = 1e-5,
    logging_dir = './logs',
    eval_steps = 100

)
with training_args.strategy.scope():
  trainer_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels = 5)

trainer = TFTrainer(
    model = trainer_model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [40]:
trainer.train()
trainer.evaluate()

{'eval_loss': 0.8767999921526227}

In [41]:
save_directory = "/save_models"

model.save_pretrained(save_directory)

tokenizer.save_pretrained(save_directory)

('/save_models/tokenizer_config.json',
 '/save_models/special_tokens_map.json',
 '/save_models/vocab.txt',
 '/save_models/added_tokens.json')

In [42]:
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)
model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)


Some layers from the model checkpoint at /save_models were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /save_models and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
test_text = test_texts[0]
test_text

'mumbai bombs movie postponed the release of a film about the mumbai (bombay) blasts in 1993 has been postponed following protests by those on trial for the bombings.  investigating the blasts which killed more than 250 people and wounded 1 000  the film black friday had been due to open across india on friday. but 36 people accused in connection with the blasts said it should not be screened until the trial is over. mumbai high court postponed the film s launch until 3 february.  black friday is based upon the novel of the same name written by journalist s hussain zaidi  which looks at the 15 explosions which rocked mumbai on 12 march 1993. director anurag kashyap said the film s release should not be delayed as the book has already been on sale for two years.  if you have not gone against the book  then how can you go against the movie   mr kashyap s lawyer mihir desai said. this is the second time that those accused in connection with the blasts have sought legal intervention regard

In [45]:
predict_input = tokenizer_fine_tuned.encode(
    test_text,
    truncation = True,
    padding = True,
    return_tensors = 'tf'
)

output = model_fine_tuned(predict_input)[0]
prediction_value = tf.argmax(output, axis = 1).numpy()[0]
prediction_value

2

### Inferencing with Pytorch

In [50]:
import torch
from transformers import DistilBertForSequenceClassification
tokenizer_fine_tuned_pt = DistilBertTokenizer.from_pretrained(save_directory)
model_fine_tuned_pt = DistilBertForSequenceClassification.from_pretrained(save_directory, from_tf = True)


All TF 2.0 model weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


In [ ]:
output = model_fine_tuned_pt(**inputs)

# Get the predicted class
predicted_class = torch.argmax(output.logits, dim=2)[0].tolist()

# Decode the predicted class
predicted_tokens = tokenizer_fine_tuned_pt.convert_ids_to_tokens(predicted_class)

print(predicted_tokens)

In [57]:
inputs = tokenizer_fine_tuned_pt(test_text, truncation = True, padding = True, return_tensors = 'pt')
output_pt = model_fine_tuned_pt(**inputs)
prediction_value_pt = torch.argmax(output.logits, dim = 2)[0].tolist()
prediction_value_pt

AttributeError: ignored